## Zadanie:

Proszę spróbować przeprowadzić taki proces regularyzacji modeli dla innego, nietrywialnego zbioru uczącego.

In [191]:
import numpy as np
import tensorflow as tf
import keras
import pandas as pd
from sklearn.model_selection import train_test_split

print ("TensorFlow version: " + tf.__version__)
print ("Keras version: " + keras.__version__)

from keras import models
from keras import layers

TensorFlow version: 2.16.1
Keras version: 3.0.5


In [192]:
import seaborn as sns
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report
LABELS= ['0', '1']

# Define the confusion matrix for the results
def show_confusion_matrix(validations, predictions, num_classes):
    matrix = metrics.confusion_matrix(validations, predictions)
    plt.figure(figsize=(num_classes, num_classes))
    hm = sns.heatmap(matrix,
                cmap='coolwarm',
                linecolor='white',
                linewidths=1,
                xticklabels=LABELS,
                yticklabels=LABELS,
                annot=True,
                fmt='d')
    plt.yticks(rotation = 0)  # Don't rotate (vertically) the y-axis labels
    hm.set_ylim(0, len(matrix))
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()

### Dataset
For this excercice I'm going to use Hotel Reservations dataset.
The goal is to predict whether the booking was cancelled or not.
https://www.kaggle.com/datasets/ahsan81/hotel-reservations-classification-dataset

### Load data

In [193]:
data = pd.read_csv('HotelReservations.csv')
data.drop(['Booking_ID'], axis=1, inplace=True)
print(data.shape)
data.head()

(36275, 18)


,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
0,2,0,1,2,Meal Plan 1,0,Room_Type 1,224,2017,10,2,Offline,0,0,0,65.00,0,Not_Canceled
1,2,0,2,3,Not Selected,0,Room_Type 1,5,2018,11,6,Online,0,0,0,106.68,1,Not_Canceled
2,1,0,2,1,Meal Plan 1,0,Room_Type 1,1,2018,2,28,Online,0,0,0,60.00,0,Canceled
3,2,0,0,2,Meal Plan 1,0,Room_Type 1,211,2018,5,20,Online,0,0,0,100.00,0,Canceled
4,2,0,1,1,Not Selected,0,Room_Type 1,48,2018,4,11,Online,0,0,0,94.50,0,Canceled


### Reshape data

In [194]:
data['booking_status'] = data['booking_status'].map({'Canceled': 1, 'Not_Canceled': 0})
data['market_segment_type'] = data['market_segment_type'].map({'Online' : 1, 'Offline' : 0})
data['type_of_meal_plan'] = data['type_of_meal_plan'].map({'Not Selected' : 0, 'Meal Plan 1' : 1, 'Meal Plan 2' : 2, 'Meal Plan 3' : 3})
data['room_type_reserved'] = data['room_type_reserved'].str[-1].astype(int)
data.head()

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
0,2,0,1,2,1,0,1,224,2017,10,2,0.0,0,0,0,65.00,0,0
1,2,0,2,3,0,0,1,5,2018,11,6,1.0,0,0,0,106.68,1,0
2,1,0,2,1,1,0,1,1,2018,2,28,1.0,0,0,0,60.00,0,1
3,2,0,0,2,1,0,1,211,2018,5,20,1.0,0,0,0,100.00,0,1
4,2,0,1,1,0,0,1,48,2018,4,11,1.0,0,0,0,94.50,0,1


### Normalize data

```python
# min-max normalization:
df=(df-df.min())/(df.max()-df.min())
# mean normalization:
df=(df-df.mean())/df.std()
```

In [195]:
data = (data - data.mean()) / data.std()
data.head()

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
0,0.298889,-0.261467,0.217398,-0.144801,0.104197,-0.178817,-0.506404,1.614874,-2.137440,0.839230,-1.555641,-1.484895,-0.162208,-0.063392,-0.087455,-1.095018,-0.788129,-0.698052
1,0.298889,-0.261467,1.365974,0.563964,-1.978307,-0.178817,-0.506404,-0.933688,0.467837,1.164974,-1.097998,0.673429,-0.162208,-0.063392,-0.087455,0.092805,0.483754,-0.698052
2,-1.628953,-0.261467,1.365974,-0.853566,0.104197,-0.178817,-0.506404,-0.980237,0.467837,-1.766723,1.419035,0.673429,-0.162208,-0.063392,-0.087455,-1.237511,-0.788129,1.432519
3,0.298889,-0.261467,-0.931177,-0.144801,0.104197,-0.178817,-0.506404,1.463590,0.467837,-0.789491,0.503751,0.673429,-0.162208,-0.063392,-0.087455,-0.097566,-0.788129,1.432519
4,0.298889,-0.261467,0.217398,-0.853566,-1.978307,-0.178817,-0.506404,-0.433285,0.467837,-1.115235,-0.525945,0.673429,-0.162208,-0.063392,-0.087455,-0.254309,-0.788129,1.432519


### Train - Validate - Test split

In [196]:
train_data, test_data, train_labels, test_labels = train_test_split(data.drop(columns = ['booking_status']), data['booking_status'], test_size=0.2, random_state=42)
val_data, test_data, val_labels, test_labels = train_test_split(test_data, test_labels, test_size=0.5, random_state=42)
test_data

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests
34727,-1.628953,-0.261467,1.365974,-0.853566,0.104197,-0.178817,-0.506404,-0.980237,0.467837,-1.766723,1.419035,-1.484895,-0.162208,-0.063392,-0.087455,-1.237511,-0.788129
21428,0.298889,-0.261467,0.217398,-0.144801,0.104197,-0.178817,-0.506404,2.382934,0.467837,-0.463746,0.160519,-1.484895,-0.162208,-0.063392,-0.087455,-0.895527,-0.788129
33234,2.226730,-0.261467,1.365974,-0.853566,2.186701,-0.178817,-0.506404,0.614069,-2.137440,0.187742,0.732572,-1.484895,-0.162208,-0.063392,-0.087455,0.999631,0.483754
11713,0.298889,-0.261467,-0.931177,-0.144801,0.104197,-0.178817,-0.506404,-0.654393,0.467837,1.490718,-1.555641,NaN,-0.162208,-0.063392,-0.087455,-0.525045,-0.788129
4977,0.298889,-0.261467,-0.931177,-0.144801,-1.978307,5.592161,-0.506404,-0.887139,0.467837,0.187742,0.846982,0.673429,-0.162208,-0.063392,-0.087455,0.956883,1.755637
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24520,0.298889,-0.261467,0.217398,-0.853566,0.104197,-0.178817,-0.506404,-0.712580,0.467837,-1.440979,1.190214,0.673429,-0.162208,-0.063392,-0.087455,1.070877,0.483754
12514,2.226730,-0.261467,1.365974,-0.853566,0.104197,-0.178817,3.065405,-0.363462,0.467837,0.839230,-0.754766,0.673429,-0.162208,-0.063392,-0.087455,1.239589,1.755637
21370,0.298889,-0.261467,-0.931177,-0.853566,0.104197,-0.178817,0.207958,1.568325,0.467837,0.187742,-0.525945,0.673429,-0.162208,-0.063392,-0.087455,-0.222390,-0.788129
15809,0.298889,-0.261467,1.365974,-0.853566,2.186701,-0.178817,-0.506404,0.369686,-2.137440,0.187742,-1.670051,-1.484895,-0.162208,-0.063392,-0.087455,-0.254309,-0.788129


### Stats

In [197]:
print(f"Positive cases in total: {data['booking_status'].sum()}, negative cases in total: {len(data['booking_status']) - data['booking_status'].sum()}, % of positive cases: {data['booking_status'].sum() / len(data['booking_status']) * 100:.2f}%")

Positive cases in total: -8.952838470577262e-13, negative cases in total: 36275.0, % of positive cases: -0.00%


## Visualization functions

In [198]:
def PlotThreeAccuracyComparison(acc1, val_acc1, acc2, val_acc2, acc3, val_acc3, lab1 = '1', lab2 = '2', lab3 = '3'):
    plt.clf()   # clear figure
    plt.rcParams['figure.figsize'] = (25.0, 5.0) # set default size of plots
    epochs = range(1, len(acc1) + 1)
    plt.plot(epochs, acc1, 'bo', label='Training accuracy for ' + lab1)
    plt.plot(epochs, val_acc1, 'b', label='Validation accuracy for ' + lab1)
    plt.plot(epochs, acc2, 'ro', label='Training accuracy for ' + lab2)
    plt.plot(epochs, val_acc2, 'r', label='Validation accuracy for ' + lab2)
    plt.plot(epochs, acc3, 'go', label='Training accuracy for ' + lab3)
    plt.plot(epochs, val_acc3, 'g', label='Validation accuracy for ' + lab3)
    plt.title('Comparison of Training and Validation Accuracies')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

def PlotThreeLossComparison(loss1, val_loss1, loss2, val_loss2, loss3, val_loss3, lab1 = '1', lab2 = '2', lab3 = '3'):
    plt.clf()   # clear figure
    plt.rcParams['figure.figsize'] = (25.0, 5.0) # set default size of plots
    epochs = range(1, len(loss1) + 1)
    plt.plot(epochs, loss1, 'bo', label='Training loss for ' + lab1)
    plt.plot(epochs, val_loss1, 'b', label='Validation loss for ' + lab1)
    plt.plot(epochs, loss2, 'ro', label='Training loss for ' + lab2)
    plt.plot(epochs, val_loss2, 'r', label='Validation loss for ' + lab2)
    plt.plot(epochs, loss3, 'go', label='Training loss for ' + lab3)
    plt.plot(epochs, val_loss3, 'g', label='Validation loss for ' + lab3)
    plt.title('Comparison of Training and Validation Losses')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()


## Create NN

In [199]:
network = models.Sequential()
network.add(layers.Dense(64, activation='relu', input_shape=(10,)))
network.add(layers.Dense(64, activation='relu'))
network.add(layers.Dense(1, activation='sigmoid'))
# network.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])


/home/lenovo/Uni/DeepNeuralNetworks-Course/.venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
